# Instructions
The following code was designed in order to track the location of a single animal across the course of a session.  After initally loading in the video, the user is able to crop the video frame using a rectangle selection tool.  A background reference frame is then specified, either by taking an average of several frames in the video, or by the user providing a short video of the same environment without an animal in it.  By comparing each frame in the video to the reference frame, the location of the animal can be tracked.  It is imperative that the reference frame of the video is not shifted from the actual video.  For this reason, if a separate video is supplied, it is best that it be acquired on the same day as the behavioral recording.  The location of the animal (it's center of mass, or COM) in x,y coordinates is then recorded, as well as the distance in pixels that the animal moves from one frame to the next. Lastly, the user can specify regions of interest in the frame (e.g. left, right) using a polygon drawing tool and record for each frame whether the animal is in the region of interest.  Options for summarizing the data are also provided. 

### Package Requirements
Please see instructions under repository README for package requirements and install instructions.

---
# 1. Load Necessary Packages

In [ ]:
%load_ext autoreload
%autoreload 2
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import BehaviorTracking_Functions as fz
import holoviews as hv
plt.gray()

---
# 2. User Sets Directory and File Information

In [ ]:
dpath = "/Users/ZP/Desktop/Videos" # directory containing file
file = "behavCam1.avi" #filename.  will take mpg and wmv files but maybe more.  only mpg1 has been extensively tested.
fps = 30 #frames per second


---
# 3. Load Video and Crop Frame if Desired
To crop video frame, select box selection tool above image (square with a plus sign).  To start drawing region to be included in analyis, double click image.  Double click again to finalize region.  If you decide to change region, it is best to rerun this cell and subsequent steps.

In [ ]:
%%output size=150

#Select output size if image is too small/large.  Code above must be first line in cell and dictates overall size
#of image, where 100 is standard.  stretch_w and stretch_h below allow you to alter relative width to height ratio.

stretch_w = .8 #Default=1. Can be used to stretch image width if needed 
stretch_h = 2 #Default=1. Can be used to stretch image height if needed 

#Load image to allow cropping
image,crop,fpath=fz.LoadAndCrop(dpath,file,stretch_w,stretch_h,cropmethod='HLine')
image

---
# 4. Define Reference Frame for Location Tracking
For location tracking to work, view of box without animal must be provided.  Below there are two ways to do this.  **Option 1** provides a method to remove the animal from the video.  This option works well provided the animal doesn't stay in the same location the whole time. Alternatively, with **Option 2**, the user can simply define a video file in the same folder that doesn't have in animal in it.  Option 1 is generally recormmended for this reason.   

### Option 1 - Create reference frame by removing animal from video
The following code takes 100 random frames across the session and creates an average of them by taking the median for each pixel.  This will remove influence of animal on any given frame.

In [ ]:
#Create Reference Frame
reference = fz.Reference(fpath,crop,f=100) #f is paramater for number of frames from video to be sampled

#Display Reference Frame
plt.figure(figsize=(20,20))
plt.title('Reference Frame',size=25)
plt.imshow(reference)

### Option 2 - User specifies video of empty box
The following code allows the user to specify a different file.  Notably, an average is still taken of multiple frames.

In [ ]:
#User selects file
altfile = 'file' #specify filename of video in dpath directory (e.g. 'Video1.mpg')

#Create Reference Frame
altpath = dpath + '/' + altfile #program sets file path from directory path and filenmae
reference = fz.Reference(altpath,crop,f=100) #f is paramater for number of frames from video to be sampled

#Display Reference Frame
plt.figure(figsize=(20,20))
plt.title('Reference Frame',size=25)
plt.imshow(reference)

---
# 5. Track Location

### 5a. Set Location Tracking Parameters
Location tracking examines the deviance of each frame in a video from the reference frame on a pixel by pixel basis.  For each frame it calculates the center of mass for these differences (COM) to define the center of the animal.  

In order to improve accuracy, the parameter loc_thresh is used to remove the influence of pixels that are minimally different from the reference frame.  For each frame relative to the reference frame, the distribution of absolute difference values across pixels is examined and only values above loc_thresh percentile are considered.  I have been using 99 and this works well.  Values can range from 0-100 and floating point numbers are fine.

The parameters use_window, window, and window_weight are employed to reduce the chance that any objects other than the animal  that might enter the frame (e.g. the hand of the experimenter) influence location tracking.  For each frame, a square window with the animal's position on the prior frame at its center is given more weight when searching for it's location (because an animal presumably can't move far in a fraction of a second).  When window_weight is set to 0, pixels outside of the window are not considered at all; at 1, they are given equal weight.  Notably, setting a low but non-zero value (e.g. 0.1) should allow the program to more rapidly find the animal if by chance it moves out of the window.

In [ ]:
loc_thresh = 99
use_window = True #True/False.  Will window surrounding prior location of animal be used
window = 100 #The lenght of one side of square window for weighting pixels around prior location of animal.  
window_weight = .2 #0-1 scale, where 0 is maximal weight of window surrounding prior locaiton.
SIGMA = 1 #this is sigma used for gaussian smoothing of image.  Used to reduce influence of frame by frame pixel jitter. 1 works well but could increased slightly if noise is detected. 

### 5b. Display Examples of Location Tracking to Confirm Threshold
In order to confirm threshold is working a subset of images is analyzed and displayed using the selected loc_thresh.  The original image is displayed on top and the difference scores are presented below.  The center of mass (COM) is pinpointed on images.  Notably, because individual frames are used, window settings are not applicable here.

In [ ]:
#number of examples
examples = 3
figsize=(20,2)

fz.LocationThresh_View(examples,figsize,fpath,reference,SIGMA,loc_thresh,crop)

### 5c. Track Location and Save Results to .csv File
For each frame the location of the animal's center of mass is recorded in x,y coordinates.  Frame-by-frame distance is also calculated in pixel units.  This data is returned in a Pandas dataframe with columns: frame, x, y, dist.  Data is saved to a .csv in the same folder as the video.

In [ ]:
location=fz.TrackLocation(fpath,reference,SIGMA,loc_thresh,crop,use_window,window,window_weight)

#Set output name and save
fpath_out = fpath[:-4] + '_LocationOutput.csv'
location.to_csv(fpath_out)
location

### 5d. Display Animal Location Across Session
Below, animals distance, x, and y coordinates across the sessin are plotted.  Smooth traces are expected in the case where they animal is tracked consistently across the session.

In [ ]:
plt.figure(figsize=(20,8))
plt.gray()
plt.title('Animal Location Across Session',size=25)
implot = plt.imshow(reference)
plt.scatter(x=location['X'], y=location['Y'], c='b', alpha=1, s=2)

#location
w, h = 800,150
dist_plot = hv.Curve((location['Frame'],location['Distance']),'Frame','Pixel Distance').opts(height=h,width=w,color='blue',title="Distance")
x_plot = hv.Curve((location['Frame'],location['X']),'Frame','X').opts(height=h,width=w,color='blue',title="X Position")
y_plot = hv.Curve((location['Frame'],location['Y']),'Frame','Y').opts(height=h,width=w,color='blue',title="Y Position")
layout=(dist_plot + x_plot + y_plot).cols(1)
layout

# 6. (Optional) Analyze Activity of Animal in Region of Interest

### 6a. User Supplies Names of Regions of Interest

In [ ]:
region_names = ["Left","Right"]

### 6b. Use Interactive Plot to Define Regions of Interest.  Supports Polygons. 
Draw regions of interest in the order you provided them.  To start drawing a region, double click on image.  Single click to add a vertex.  Double click to close polygon.

In [ ]:
%%output size=200
#Select output size
stretch_w = .8 #Default=1. Can be used to stretch axis if needed 
stretch_h = 2 #Default=1. Can be used to stretch axis if needed 

plot,poly_stream = fz.ROI_plot(reference,region_names,stretch_w,stretch_h)
plot

### 6c. Save Amended Output File with Locations as .csv

In [ ]:
location_wROI = fz.ROI_Location(reference,poly_stream,region_names,location)

#Set output name and save
fpath_out = fpath[:-4] + '_LocationOutput.csv'
location_wROI.to_csv(fpath_out)
location_wROI